• 1) Consider the karate club graph in the networkx distribution used in Lab4
G = nx.karate_club_graph()
• Try to implement the Baryentric heuristic to sort nodes in the arc diagram and use
nxviz to display the arc diagram sorted according to the heuristic and with nodes
colored according to the node degree
• Remember: the algorithm is not guaranteed to converge, so use a limited number of
iterations

In [ ]:
                                                                                                                                                                                                                                                                                                  import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import nxviz as nv
import numpy as np

G = nx.karate_club_graph()

def barycentric_heuristic(G, iterations=15):
  nodes=list(G.nodes())
  barycenters={}
  for n in nodes:
    neighbors=list(G.neighbors(n))
    if len(neighbors)>0:
      neighbors_index=[nodes.index(neighbor) for neighbor in neighbors]
      barycenter=np.mean(neighbors_index)
    else:
      barycenter=0
    barycenters[n]=barycenter
  nodes.sort(key=lambda n: barycenters[n])
  return nodes

ordered_nodes=barycentric_heuristic(G)

node_order={}
for index, node in enumerate(ordered_nodes):
  node_order[node]=index
nx.set_node_attributes(G, node_order, 'order')

node_colors=[]
for node in G.nodes():
  degree=G.degree(node)
  node_colors.append(degree)

arcplot = nv.arc(G, node_color=node_colors, node_order=list(node_order.values()))

plt.figure(figsize=(10, 5))
plt.show()



TypeError: base() got an unexpected keyword argument 'node_order'

• Load the spotify 30K songs dataset used in HW1. Consider the following tasks
(and related target data)
• Using Plotly, create a treemap, representing the hierarchy of the playlist_genres and
subgenres, with box sizes mapped from the number of items, and color mapped from the
average popularity. Use a non-default, perceptually uniform sequential colormap and set
the color range mapped to the whole range of potential values of the average popularity
(0-100). Visualize the same hierarchy tree as a node-link diagram with node size
proportional to the number of items, the color proportional to the average popularity
• Create a simple Dash app with the possibility of selecting the songs of different decades
(1950-1960, 1961-1970,…) with a menu and the type of chart (treemap or sunburst) with
another menu. The apps should show with the selected chart type the tree representation
of the hierarchy: all->genres->subgenres

In [ ]:
import plotly.express as px
import pandas as pd


songs = pd.read_csv('spotify_songs.csv')

avg_popularity = songs.groupby(['playlist_genre', 'playlist_subgenre'])['track_popularity'].mean().reset_index()
avg_popularity['num_items'] = songs.groupby(['playlist_genre', 'playlist_subgenre'])['track_popularity'].count().values

fig = px.treemap(avg_popularity,
                 path=['playlist_genre', 'playlist_subgenre'],
                 values='num_items',
                 color='track_popularity',
                 color_continuous_scale='RdBu',
                 range_color=[0, 100],
                 title='Treemap of Genres and Subgenres')
fig.show()

fig2= px.sunburst(avg_popularity,
                  path=['playlist_genre', 'playlist_subgenre'],
                  values='num_items',
                  color='track_popularity',
                  color_continuous_scale='RdBu',
                  range_color=[0, 100],
                  title='Sunburst of Genres and Subgenres')
fig2.show()




In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash
import plotly.express as px
import pandas as pd

songs=pd.read_csv('spotify_songs.csv')
app=dash.Dash(__name__)

app.layout=html.Div([
    dcc.Dropdown(
        id='decade-dropdown',
        options=[{'label':f' {decade}--{decade+9}', 'value':f'{decade}-{decade+9}'}
                 for decade in range(1950,2020,10)],
        value='1950-1960'
    ),
    dcc.Dropdown(
        id='chart-type-dropdown',
        options=[{'label': 'Treemap', 'value': 'treemap'},
                 {'label': 'Sunburst', 'value': 'sunburst'}],
        value='treemap'
    ),
    dcc.Graph(id='chart')
])

@app.callback(
    Output('chart', 'figure'),
    [Input('decade-dropdown', 'value'),
     Input('chart-type-dropdown', 'value')]
)
def update_chart(decade, chart_type):
  start_year, end_year=map(int, decade.split('-'))
  filtered_songs = songs['track_album_release_date'].str[:4].astype(int).between(start_year, end_year)
  avg_popularity=filtered_songs.groupby(['playlist_genre', 'playlist_subgenre'])['track_popularity'].mean().reset_index()
  avg_popularity['num_items']=filtered_songs.groupby(['playlist_genre', 'playlist_subgenre'])['track_popularity'].count().values
  if chart_type=='treemap':
    fig=px.treemap(avg_popularity,
                 path=['playlist_genre', 'playlist_subgenre'],
                 values='num_items',
                 color='track_popularity',
                 color_continuous_scale='RdBu',
                 range_color=[0, 100],
                 title='Treemap of Genres and Subgenres')
  else:
    fig= px.sunburst(avg_popularity,
                  path=['playlist_genre', 'playlist_subgenre'],
                  values='num_items',
                  color='track_popularity',
                  color_continuous_scale='RdBu',
                  range_color=[0, 100],
                  title='Sunburst of Genres and Subgenres')
  return fig
app.run_server(mode='inline')

<IPython.core.display.Javascript object>